<a href="https://colab.research.google.com/github/hannnzu/Lawu-Geo-Tracker/blob/main/Lawu_Geo_Tracker.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# TEAM PROJECT DATA ENGINEERING KELOMPOK 8

***
**Open Data Source Analysis & Planning Project**

**Project Name**: Lawu Geo-Tracker: Integrasi Data Spasial & Elevasi Jalur Pendakian Gunung Lawu

**Created By**: Data Engineering Team 8

**Date**: February 19, 2026
**Version**: 1.0

***

**1. Executive Summary**

**1.1 Project Overview**


*   **Tujuan Project**: Membangun data geospasial untuk memetakan jalur pendakian Gunung Lawu beserta elevasi (ketinggian) dan estimasi waktu tempuh.
*   **Scope Project**: Integrasi data spasial rute dari OpenStreetMap, data topografi (elevasi), dan data aktivitas pendakian historis (GPX) pada 3 jalur utama (cemoro sewu, cemoro kandang, dan candi cetho)
*   **Expected Outcomes**: Dashboard analitik dan sistem rekomendasi rute
*   **Timeline**: 3 bulan (Februari - April 2026)

***

**2. Data Source Analysis**

**2.1 Data Pemerintah ([inarisk.bnpb.go.id](http://inarisk.bnpb.go.id/))**

**Source Details**
*   **Dataset Name**: Peta Indeks Rawan Bencana Area Gunung Lawu.
*   **URL/Access Point**: [inarisk.bnpb.go.id](http://inarisk.bnpb.go.id/)
*   **Data Owner**: Badan Nasional Penanggulangan Bencana
*   **Update Frequency**: Yearly

**Data Analysis**
*   **Format Data**: GeoJSON, WMS
*   **Volume Data**: ~15MB
*   **Time Coverage**: Indeks Risiko Bencana Terbaru (2024 - 2025)
*   **Data Quality**:
    * Completeness: 100% (Mencakup seluruh tutupan lahan di area Gunung Lawu).
    * Accuracy: Tinggi (Data resmi negara yang diverifikasi oleh para ahli kebencanaan dan GIS).
    * Consistency: Standar atribut spasial nasional.
    * Timeliness: Diperbarui secara berikala.

**2.2 Dataset wikiloc**

**Source Details**
*   **Dataset Name**: Jalur pendakian gunung lawu
*   **URL/Access Point**: [https://www.wikiloc.com/wikiloc/](https://www.wikiloc.com/wikiloc/map.do?sw=-89.999%2C-179.999&ne=89.999%2C179.999&q=Gunung%20Lawu%20Cemoro%20Sewu&fitMapToTrails=1&page=1)
*   **Data Owner**: Hiker comunity
*   **Update Frequency**: January 19, 2026

**Data Analysis**
*   **Format Data**: GPX
*   **Size & Dimensions**: ~2MB & ~4 coloumns dengan ~600 baris titik rekam.
*   **Data Fields**:
    * lat (latitude): Koordinat lintang.
    * lon (longitude): Koordinat bujur.
    * ele (elevation): Ketinggian dari permukaan laut (MDPL).
    * time (timestamp): Waktu dan tanggal titik WPS.
    * metadata: informasi dari pembuat track (seperti nama rute, author, ataupun link sumber)

*   **Quality Metrics**:
    * Missing Values: kemungkinan 0% karena semua data memiliki titik perekaman data lintang, bujur, serta ketinggian.
    * Data Types:
      * lat & lan: Float.
      * ele: Float.
      * time: Datetime.
      * metadata: String.
    * Consistency: Medium (data dibuat dari alat GPS ataupun device yang berbeda-beda)
    * Documentation Quality: Fair (GPX format standar berbasis XML yang bersifat self-describing).

**Public APIs**

**Source Details**
*   **API Name**: Overpass API (OpenStreetMap)
*   **Endpoint URL**: [overpass-api.de/api/interpreter](http://overpass-api.de/api/interpreter)
*   **Provider**: OpenStreetMap Foundation
*   **Authentication Method**:No Auth (Open Public API)

**API Analysis**
*   **Response Format**: JSON (OSM JSON) / GeoJSON
*   **Rate Limits**: ~10.000 requests / day
*   **Reliability**: 99.9% uptime
*   **Documentation Quality**:Very good.
*   **Cost**: Free

**2.4 Open Research Data**

**Source Details**
*   **Dataset Name**: ALOS World 3D - 30m (AW3D30) Digital Elevation Model.
*   **Repository**: OpenTopography
*   **Research Institution**: Japan Aerospace Exploration Agency (JAXA)
*   **Publication Date**: Version 3.2 (Januari 2021)

**Data Analysis**
*   **Format & Structure**: GeoTIFF
*   **Data Volume**: < 10 MB.
*   **Data Quality**:
    * Accuracy: very high (resolusi spasial 30 meter per piksel).
*   **Citation Requirements**: Japan Aerospace Exploration Agency (2021). ALOS World 3D 30 meter DEM. V3.2, Jan 2021. Distributed by OpenTopography.

**2.5 Data Cuaca**

**Source Details**
*   **API Name**: OpenWeatherMap Current Weather Data API
*   **Endpoint URL**: [api.openweathermap.org/data/2.5/weather](http://api.openweathermap.org/data/2.5/weather)
*   **Provider**: OpenWeather Ltd.
*   **Authentication Method**: API Key

**API Analysis**
*   **Response Format**: JSON
*   **Rate Limits**: 1.000 API calls/day
*   **Reliability**: 99.9% uptime
*   **Documentation Quality**: Medium-high.
*   **Cost**: Free.

In [5]:
!wget https://raw.githubusercontent.com/hannnzu/Lawu-Geo-Tracker/refs/heads/main/DATA/mount-lawu-via-cemoro-sewu.gpx

--2026-02-27 06:57:14--  https://raw.githubusercontent.com/hannnzu/Lawu-Geo-Tracker/refs/heads/main/DATA/mount-lawu-via-cemoro-sewu.gpx
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.110.133, 185.199.111.133, 185.199.108.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.110.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 290792 (284K) [text/plain]
Saving to: ‘mount-lawu-via-cemoro-sewu.gpx’

mount-lawu-via-cemo 100%[===================>] 283.98K  --.-KB/s    in 0.03s   

2026-02-27 06:57:14 (8.83 MB/s) - ‘mount-lawu-via-cemoro-sewu.gpx’ saved [290792/290792]



In [27]:
import pandas as pd
import gpxpy
import gpxpy.gpx

gpx_file_path = "/content/Gunung Lawu Track.gpx"
gpx_file_path = "/content/gunung-lawu-via-candi-cetho-karanganyar-jawa-tengah-agustus-.gpx"
gpx_file_path = "/content/jalur-pendakian-gunung-lawu-via-cemoro-kandang.gpx"
gpx_file_path = "/content/lawu-via-cemoro-sewu-6-2-km-run-segment-in-on-strava.gpx"
gpx_file_path = "/content/mount-lawu-via-cemoro-sewu.gpx"


# Open and parse the GPX file
with open(gpx_file_path, 'r') as gpx_file:
    gpx = gpxpy.parse(gpx_file)

data = []

for track in gpx.tracks:
    for segment in track.segments:
        for point in segment.points:
            data.append({
                'latitude': point.latitude,
                'longitude': point.longitude,
                'elevation': point.elevation,
                'time': point.time
            })

lawu_track = pd.DataFrame(data)

print("Shape:", lawu_track.shape)
print("Index:", lawu_track.index)


Shape: (2900, 4)
Index: RangeIndex(start=0, stop=2900, step=1)


In [28]:
lawu_track.head()

,latitude,longitude,elevation,time
0,-7.664584,111.191485,1912.046,2018-03-29 03:10:55+00:00
1,-7.664590,111.191485,1912.088,2018-03-29 03:10:56+00:00
2,-7.664579,111.191493,1913.316,2018-03-29 03:11:00+00:00
3,-7.664516,111.191522,1912.008,2018-03-29 03:11:07+00:00
4,-7.664436,111.191518,1912.446,2018-03-29 03:11:30+00:00


In [23]:
lawu_track.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2900 entries, 0 to 2899
Data columns (total 4 columns):
 #   Column     Non-Null Count  Dtype                        
---  ------     --------------  -----                        
 0   latitude   2900 non-null   float64                      
 1   longitude  2900 non-null   float64                      
 2   elevation  2900 non-null   float64                      
 3   time       2900 non-null   datetime64[ns, SimpleTZ('Z')]
dtypes: datetime64[ns, SimpleTZ('Z')](1), float64(3)
memory usage: 90.8 KB


In [26]:
lawu_track.index

RangeIndex(start=0, stop=652, step=1)

In [29]:
lawu_track['elevation']

,elevation
0,1912.046
1,1912.088
2,1913.316
3,1912.008
4,1912.446
...,...
2895,1918.108
2896,1917.525
2897,1917.146
2898,1917.160


In [33]:
lawu_track['elevation'].unique()

array([1912.046, 1912.088, 1913.316, ..., 1917.146, 1917.16 , 1916.431])

In [39]:
lawu_track[['latitude', 'longitude']].value_counts()

,,count
latitude,longitude,
-7.627365,111.193511,2
-7.625317,111.196738,2
-7.625306,111.196757,2
-7.625282,111.196698,2
-7.632613,111.196143,1
...,...,...
-7.643016,111.195018,1
-7.642998,111.195034,1
-7.642962,111.195027,1


In [44]:
!pip install geopandas
import geopandas

gpkg_file_path = "/content/jalur_lawu_lengkap.gpkg"

try:
    lawu_geojson = geopandas.read_file(gpkg_file_path)
    print("GeoPackage file read successfully!")
    print("Shape:", lawu_geojson.shape)
    print("Columns:", lawu_geojson.columns)
    print(lawu_geojson.head())
except Exception as e:
    print(f"Error reading GeoPackage file: {e}")
    print("Please ensure the file exists and is a valid GeoPackage.")


GeoPackage file read successfully!
Shape: (604, 6)
Columns: Index(['osmid', 'y', 'x', 'street_count', 'junction', 'geometry'], dtype='object')
       osmid         y           x  street_count junction  \
0  275874772 -7.661931  111.153876             3            
1  275874775 -7.662126  111.154837             3            
2  275874787 -7.662877  111.158751             3            
3  275874857 -7.666510  111.170594             3            
4  275874891 -7.664624  111.175493             4      yes   

                     geometry  
0  POINT (111.15388 -7.66193)  
1  POINT (111.15484 -7.66213)  
2  POINT (111.15875 -7.66288)  
3  POINT (111.17059 -7.66651)  
4  POINT (111.17549 -7.66462)  


/usr/local/lib/python3.12/dist-packages/pyogrio/geopandas.py:382: UserWarning: More than one layer found in 'jalur_lawu_lengkap.gpkg': 'nodes' (default), 'edges'. Specify layer parameter to avoid this warning.
  result = read_func(


In [45]:
display(lawu_geojson)

,osmid,y,x,street_count,junction,geometry
0,275874772,-7.661931,111.153876,3,,POINT (111.15388 -7.66193)
1,275874775,-7.662126,111.154837,3,,POINT (111.15484 -7.66213)
2,275874787,-7.662877,111.158751,3,,POINT (111.15875 -7.66288)
3,275874857,-7.666510,111.170594,3,,POINT (111.17059 -7.66651)
4,275874891,-7.664624,111.175493,4,yes,POINT (111.17549 -7.66462)
...,...,...,...,...,...,...
599,13114536199,-7.650636,111.158136,3,,POINT (111.15814 -7.65064)
600,13114536203,-7.650984,111.158496,3,,POINT (111.1585 -7.65098)
601,13114536319,-7.669685,111.148837,1,,POINT (111.14884 -7.66968)
602,13114536530,-7.660755,111.176137,3,,POINT (111.17614 -7.66075)
